# * VINSIGHT : Gross Adds

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000101CS' --Prepaid Gross Adds
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'TB1S000101CG' --Prepaid Gross Adds : TMH - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds : DTAC - GEO Channel
        
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'B2S020100CS' --Postpaid Gross Adds B2B
        , 'TB2S020100CS' --Postpaid Gross Adds B2B : TMH
        , 'DB2S020100CS' --Postpaid Gross Adds B2B : DTAC
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'TB2S010100CG' --Postpaid Gross Adds B2C : TMH - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C : DTAC - GEO Channel
        , 'B2S020100CG' --Postpaid Gross Adds B2B - GEO Channel
        , 'TB2S020100CG' --Postpaid Gross Adds B2B : TMH - GEO Channel
        , 'DB2S020100CG' --Postpaid Gross Adds B2B : DTAC - GEO Channel
        
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB3S000102D1CS' --TOL Gross Adds Connected : Consumer (Install Location)
        , 'TB3S000102D1CG' --TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000103CG' --TOL Gross Adds Connected : DataService - GEO Channel
        , 'TB3S000103D1CS' --TOL Gross Adds Connected : DataService (Install Location)
        , 'TB3S000103D1CG' --TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB3S000105CG' --TOL Gross Adds Registration : DataService - GEO Channel
        
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001004CG' --TVS CMDU Gross Adds - GEO Channel
        , 'TB4S001400CS' --TVS Now Gross Adds
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-06-06, 10:51:33

DataFrame: 5304 rows, 13 columns


In [3]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily (All & Geo)

In [4]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


In [5]:
''' Gross Adds(All Channel) Daily '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_T'] = np.where(ga_daily_df['METRIC_CD']=='TB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_D'] = np.where(ga_daily_df['METRIC_CD']=='DB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S010100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_REG_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000104CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_CONN'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_CONN(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102D1CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_REG_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000105CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103D1CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)
# ga_daily_df['TVS_CMDU'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001004CS', ga_daily_df['P'], 0)

# ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0,0,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,0,396,0,0,0,0,"2,603","1,822",0
2,202506,20250603,2025-06-06 03:26:41,"57,933","26,621","31,312","6,496","4,770","1,726","2,618","1,733",107
3,202506,20250602,2025-06-06 03:26:41,"59,959","27,556","32,403","5,805","4,217","1,588","2,515","1,696",55
4,202506,20250601,2025-06-06 03:26:41,"64,614","28,229","36,385","5,571","4,603",968,"2,008","1,669",83
5,202505,20250531,2025-06-06 03:26:41,"71,873","30,251","41,622","5,697","3,944","1,753","2,127","1,696",144
6,202505,20250530,2025-06-06 03:26:41,"68,285","29,540","38,745","5,203","3,882","1,321","2,188","1,650",151
7,202505,20250529,2025-06-06 03:26:41,"61,793","28,036","33,757","4,821","3,543","1,278","2,409","1,572",146
8,202505,20250528,2025-06-06 03:26:41,"54,979","25,115","29,864","4,939","3,549","1,390","2,296","1,674",121
9,202505,20250527,2025-06-06 03:26:41,"53,237","24,201","29,036","5,542","4,209","1,333","2,348","1,613",110


In [6]:
''' Gross Adds(Geo Channel) Daily '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_geo_daily_df['PRE'] = np.where(ga_geo_daily_df['METRIC_CD']=='B1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C'] = np.where(ga_geo_daily_df['METRIC_CD']=='B2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB2S010100CG', ga_geo_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_daily_df['TOL_REG_CON'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000104CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TOL_CONN'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_CONN(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_REG_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000105CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TVS_NOW'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001400CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TVS_CMDU'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001004CG', ga_geo_daily_df['P'], 0)

# ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

mod_col_list = ga_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_daily_df[col] = ga_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0,0,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,0,396,0,0,0,0,"1,967","1,461",0
2,202506,20250603,2025-06-06 03:26:41,"57,606","26,554","31,052","2,535","2,025",510,"2,019","1,390",107
3,202506,20250602,2025-06-06 03:26:41,"59,608","27,491","32,117","2,367","1,863",504,"1,964","1,343",55
4,202506,20250601,2025-06-06 03:26:41,"64,160","28,133","36,027","2,761","2,152",609,"1,588","1,239",83
5,202505,20250531,2025-06-06 03:26:41,"71,591","30,148","41,443","2,180","1,794",386,"1,667","1,197",143
6,202505,20250530,2025-06-06 03:26:41,"67,853","29,398","38,455","2,083","1,700",383,"1,591","1,274",151
7,202505,20250529,2025-06-06 03:26:41,"61,538","27,924","33,614","1,860","1,545",315,"1,777","1,276",145
8,202505,20250528,2025-06-06 03:26:41,"54,358","25,029","29,329","1,947","1,577",370,"1,778","1,325",121
9,202505,20250527,2025-06-06 03:26:41,"52,867","24,102","28,765","2,293","1,838",455,"1,835","1,259",110


## Products Summary

### Prep Monthly Data

In [25]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [26]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505.0


### Prepaid

In [10]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Prepaid,B1S000101CS,Prepaid Gross Adds,2025-06-06 03:26:41,"1,912,792","1,912,792","1,649,295","1,649,295","1,624,612"
1,202505,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-06-06 03:26:41,"1,000,600","1,000,600","972,821","972,821","961,451"
2,202505,Prepaid,TB1S000101CS,Prepaid Gross Adds : TMH,2025-06-06 03:26:41,"912,192","912,192","676,474","676,474","663,161"


In [11]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Prepaid,B1S000101CG,Prepaid Gross Adds - GEO Channel,2025-06-06 03:26:41,0,"1,900,245","1,641,183","1,641,183","1,616,512"
1,202505,Prepaid,DB1S000101CG,Prepaid Gross Adds : DTAC - GEO Channel,2025-06-06 03:26:41,0,"991,701","964,902","964,902","953,544"
2,202505,Prepaid,TB1S000101CG,Prepaid Gross Adds : TMH - GEO Channel,2025-06-06 03:26:41,0,"908,544","676,281","676,281","662,968"


### Postpaid

In [12]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2S010100CS,Postpaid Gross Adds B2C,2025-06-06 03:26:41,"182,712","182,712","141,121","141,121","141,123"
1,202505,Postpaid,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-06-06 03:26:41,"47,763","47,763","47,483","47,483","47,484"
2,202505,Postpaid,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-06-06 03:26:41,"134,949","134,949","93,638","93,638","93,640"


In [13]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2S010100CG,Postpaid Gross Adds B2C - GEO Channel,2025-06-06 03:26:41,0,"78,551","78,519","77,254","75,875"
1,202505,Postpaid,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,2025-06-06 03:26:41,0,"13,842","13,815","13,815","13,815"
2,202505,Postpaid,TB2S010100CG,Postpaid Gross Adds B2C : TMH - GEO Channel,2025-06-06 03:26:41,0,"64,709","64,704","64,704","64,706"


In [14]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2S020100CS,Postpaid Gross Adds B2B,2025-06-06 03:26:41,"29,161","29,161","7,470","7,128","6,956"
1,202505,Postpaid,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-06-06 03:26:41,"9,383","9,383","3,663","3,663","3,663"
2,202505,Postpaid,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-06-06 03:26:41,"19,778","19,778","3,824","3,933","3,936"


In [15]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2S020100CG,Postpaid Gross Adds B2B - GEO Channel,2025-06-06 03:26:41,0,"4,475","4,455","4,048","4,149"
1,202505,Postpaid,DB2S020100CG,Postpaid Gross Adds B2B : DTAC - GEO Channel,2025-06-06 03:26:41,0,683,683,683,683
2,202505,Postpaid,TB2S020100CG,Postpaid Gross Adds B2B : TMH - GEO Channel,2025-06-06 03:26:41,0,"3,792","3,816","3,925","3,928"


### TOL

In [16]:
''' TOL Registration : Consumer (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Consumer')]
# tol_reg_con_df = tol_reg_con_df.loc[~tol_reg_con_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_con_df = tol_reg_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-06-06 03:26:41,"72,979","72,979","56,159","56,159","55,643"
1,202505,TOL,TB3S000104CG,TOL Gross Adds Registration : Consumer - GEO C...,2025-06-06 03:26:41,0,"56,167","56,149","56,149","55,633"


In [17]:
''' TOL Registration : DataService (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('DataService')]
# tol_reg_dsv_df = tol_reg_dsv_df.loc[~tol_reg_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_dsv_df = tol_reg_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-06-06 03:26:41,"1,932","1,932",40,40,40
1,202505,TOL,TB3S000105CG,TOL Gross Adds Registration : DataService - GE...,2025-06-06 03:26:41,0,38,38,38,38


In [18]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-06 03:26:41,"52,734","52,734","41,427","41,427","41,039"
1,202505,TOL,TB3S000102D1CS,TOL Gross Adds Connected : Consumer (Install L...,2025-06-06 03:26:41,"52,734","52,734","52,727","52,727","52,727"


In [19]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000102CG,TOL Gross Adds Connected : Consumer - GEO Channel,2025-06-06 03:26:41,0,"41,428","41,424","41,424","41,036"
1,202505,TOL,TB3S000102D1CG,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-06-06 03:26:41,0,"41,428","41,428","41,428","41,428"


In [20]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-06-06 03:26:41,"1,204","1,204",51,51,51
1,202505,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-06-06 03:26:41,"1,204","1,204",51,51,51


In [21]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-06-06 03:26:41,"1,204","1,204",51,51,51
1,202505,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-06-06 03:26:41,"1,204","1,204",51,51,51


### TVS

In [22]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TVS,TB4S001400CS,TVS Now Gross Adds,2025-06-06 03:26:41,"2,806","2,806","2,797","2,828","2,833"
1,202505,TVS,TB4S001400CG,TVS Now Gross Adds - GEO Channel,2025-06-06 03:26:41,0,"2,800","2,796","2,827","2,832"


In [23]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TVS,TB4S001004CS,TVS CMDU Gross Adds,2025-06-06 03:26:41,27,27,28,28,28
1,202505,TVS,TB4S001004CG,TVS CMDU Gross Adds - GEO Channel,2025-06-06 03:26:41,0,1,1,1,1
